## Accessing results stored in MLflow

In [1]:
import mlflow
import os
from omegaconf import OmegaConf

# Config file
cfg = OmegaConf.load("..\config\main.yaml")

# helper function
def get_table(df):
    df['Duration'] = (df.end_time - df.start_time).dt.total_seconds().round(1)
    df = df[['tags.mlflow.runName', 'Duration', 'metrics.mape', 'metrics.mae', 'metrics.rmse', 'metrics.r2']].head(9)
    df.columns = ['Models', 'Duration(s)', 'MAPE', 'MAE', 'RMSE', 'R2']
    return df.sort_values('MAPE')

# Set environment variables and MLflow tracking URI from configuration
os.environ["MLFLOW_TRACKING_USERNAME"] = cfg.mlflow.username
os.environ["MLFLOW_TRACKING_PASSWORD"] = cfg.mlflow.password
mlflow.set_tracking_uri(cfg.mlflow.uri)


In [ ]:
# model selection metrics
exp_select_models = mlflow.get_experiment_by_name('select_models_null')
select_models = mlflow.search_runs(exp_select_models.experiment_id)

In [10]:
# best model from hyperopt
exp_best_model = mlflow.get_experiment_by_name('best_model')
best_model = mlflow.search_runs(exp_best_model.experiment_id)

In [11]:
# final evaluation
exp_final_evaluation = mlflow.get_experiment_by_name('final_evaluation')
final_evaluation = mlflow.search_runs(exp_final_evaluation.experiment_id).head(1)

In [12]:
get_table(select_models)

,Models,Duration(s),MAPE,MAE,RMSE,R2
1,XGBoost,1.5,0.03,0.26,0.33,0.39
7,GradientBoosting,1.7,0.03,0.28,0.35,0.32
0,LightGBM,1.5,0.04,0.29,0.37,0.23
2,Ridge,1.4,0.04,0.36,0.56,-1.19
6,HistGradientBoosting,3.2,0.04,0.29,0.37,0.23
8,Random Forest,2.2,0.04,0.31,0.38,0.20
3,Lasso,1.5,0.05,0.44,0.55,-0.63
5,ElasticNet,1.3,0.05,0.44,0.55,-0.62
4,Linear,1.4,0.06,0.53,1.53,-31.13


In [6]:
get_table(best_model)

,Models,Duration(s),MAPE,MAE,RMSE,R2
0,hyperopt,71.3,0.03,0.26,0.34,0.38


In [7]:
get_table(final_evaluation)

,Models,Duration(s),MAPE,MAE,RMSE,R2
0,evaluation,0.7,0.02,0.15,0.21,0.62
